In [1]:
import numpy as np
from sympy import *
from sympy.solvers import solve
init_printing(use_unicode=True)

def num_sym(p: int) -> int:
    return int(p * (p + 1) / 2)

def solve_lyapunov_cov(M: Matrix, C: Matrix, p: int) -> Matrix:
    Sigma = MatrixSymbol("Sigma", p, p).as_explicit()
    Sigma_solved = solve(M*Sigma + Sigma*M.T + C, Sigma)
    scheme = [[Sigma_solved[Sigma[i,j]] for i in range(p)] for j in range(p)]
    Sigma_new = Matrix(scheme)
    return Sigma_new

def create_A_Sigma(cov: Matrix, p: int) -> Matrix:
    row_num = num_sym(p=p)
    col_num = int(p * p)
    A_Sigma = MatrixSymbol("A", row_num, col_num).as_mutable()
    for l in range(p):
        for k in range(l+1):
            for i in range(p):
                for j in range(p):
                    if (j != k) & (j != l):
                        A_Sigma[(k * p) + (l - num_sym(p=k)), i * p + j] = 0
                    elif (j == k) & (k != l):
                        A_Sigma[(k * p) + (l - num_sym(p=k)), i * p + j] = cov[l, i]
                    elif (j == l) & (l != k):
                        A_Sigma[(k * p) + (l - num_sym(p=k)), i * p + j] = cov[k, i]
                    elif (j == k) & (j == l):
                        A_Sigma[(k * p) + (l - num_sym(p=k)), i * p + j] = 2 * cov[j, i]
    return A_Sigma

def create_A(A_Sigma: Matrix, mean: Matrix, p: int) -> Matrix:
    identity = mean[0] * eye(p)
    for i in range(1, p):
        identity = Matrix.hstack(identity, mean[i] * eye(p))
    return Matrix.vstack(A_Sigma, identity)

def create_c(vech_C: Matrix, b: float, index: int, p: int) -> Matrix:
    unit_vector = eye(p).col(index)
    return Matrix. vstack(vech_C, - b * unit_vector)

Problem: Nummerierung

In [ ]:
p = 4
num_eq = num_sym(p=p) + p
num_seed = 500
d = 0.9 # Probability of edges in graph

m11, m12, m13, m14 = symbols('m11'), symbols('m12'), symbols('m13'), symbols('m14')
m21, m22, m23, m24 = symbols('m21'), symbols('m22'), symbols('m23'), symbols('m24')
m31, m32, m33, m34 = symbols('m31'), symbols('m32'), symbols('m33'), symbols('m34')
m41, m42, m43, m44 = symbols('m41'), symbols('m42'), symbols('m43'), symbols('m44')
M = Matrix([[m11, m12, m13, m14], [m21, m22, m23, m24], [m31, m32, m33, m34], [m41, m42, m43, m44]])

C = 2 * eye(p) # volatility matrix

vec_M = Matrix([[M[j, i]] for i in range(p) for j in range(p)]) # j faster than i
vech_C = Matrix([[C[i, j]] for i in range(p) for j in range(i, p)])

# Intervention
index_intervention = [0, 1]
b_1 = 2
b_2 = 3
unit_v_1 = eye(p).col(index_intervention[0])
unit_v_2 = eye(p).col(index_intervention[1])

estimates = np.empty(shape=(num_seed, num_eq), dtype=object)
statistics = np.empty(shape=(num_seed*num_eq, 2), dtype=float)

for index_1 in range(num_eq):
    for index_2 in range(index_1 + 1, num_eq):
        for seed in range(1, num_seed + 1):
            np.random.seed(seed=seed)

            bernoulli_matrix = np.random.binomial(1, d, (p, p))
            normal_matrix = np.random.normal(0, 1, (p, p))

            temp_M = bernoulli_matrix * normal_matrix
            for i in range(p): # adjust diagonal entries s.t. M stable
                row_sum = np.sum(np.abs(temp_M[i, :])) - np.abs(temp_M[i, i])
                temp_M[i, i] = - row_sum - np.abs(normal_matrix[i,i])

            concrete_M = Matrix(temp_M)

            cov = solve_lyapunov_cov(M=concrete_M, C=C, p=p)
            mean_1 = b_1 * concrete_M.inv() * unit_v_1
            mean_2 = b_2 * concrete_M.inv() * unit_v_2

            A_Sigma = create_A_Sigma(cov=cov, p=p)
            A_Sigma = create_A(A_Sigma=A_Sigma, mean=mean_1, p=p)
            A = create_A(A_Sigma=A_Sigma, mean=mean_2, p=p)

            vech_C = create_c(vech_C=vech_C, b=b_1, index=index_intervention[0], p=p)
            c = create_c(vech_C=vech_C, b=b_2, index=index_intervention[1], p=p)

            A.row_del(index_1)
            A.row_del(index_2)
            c.row_del(index_1)
            c.row_del(index_2)
            temp = A * vec_M + c

            if det(A) != 0:
                # M_est = M.subs(solve(A * vec_M + c, vec_M))
                M_est = M.subs(solve(temp, vec_M))
                M_est_np = matrix2numpy(M_est).astype(float).round(3)
                M_true = matrix2numpy(concrete_M).astype(float).round(3)
                frob = np.linalg.norm(M_est_np - M_true)

                estimates[seed - 1, index] = M_est
                statistics[num_seed * index + (seed - 1), 1] = frob
            
            statistics[num_seed * index + (seed - 1), 0] = det(A)